In [1]:
!pip install pandas
import pandas as pd


Defaulting to user installation because normal site-packages is not writeable


In [5]:
df = pd.read_csv("Dataset_5971.csv")
df.head()

,LABEL,TEXT,URL,EMAIL,PHONE
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...,No,No,No
1,ham,What's up? Do you want me to come online? If y...,No,No,No
2,ham,So u workin overtime nigpun?,No,No,No
3,ham,"Also sir, i sent you an email about how to log...",No,No,No
4,Smishing,Please Stay At Home. To encourage the notion o...,No,No,No


In [13]:
df.describe()

,LABEL,TEXT,URL,EMAIL,PHONE
count,5971,5971,5971,5971,5971
unique,5,5949,2,2,2
top,ham,The greatest test of courage on earth is to be...,No,No,No
freq,4844,2,5764,5952,5127


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5971 entries, 0 to 5970
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   5971 non-null   object
 1   TEXT    5971 non-null   object
 2   URL     5971 non-null   object
 3   EMAIL   5971 non-null   object
 4   PHONE   5971 non-null   object
dtypes: object(5)
memory usage: 233.4+ KB


In [11]:
df['LABEL'].unique()

array(['ham', 'Smishing', 'spam', 'Spam', 'smishing'], dtype=object)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['TEXT']) 
y = df['LABEL']  


In [19]:
X

<5971x9556 sparse matrix of type '<class 'numpy.float64'>'
	with 81764 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [ ]:
model = RandomForestClassifier()
model

RandomForestClassifier()

In [25]:
model.fit(X_train, y_train)


RandomForestClassifier()

In [26]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

    Smishing       0.82      0.79      0.81       113
        Spam       0.00      0.00      0.00         6
         ham       0.96      1.00      0.98       974
    smishing       0.00      0.00      0.00         6
        spam       0.73      0.51      0.60        96

    accuracy                           0.93      1195
   macro avg       0.50      0.46      0.48      1195
weighted avg       0.92      0.93      0.92      1195

Accuracy: 0.9305439330543933


In [30]:
X_test

<1195x9556 sparse matrix of type '<class 'numpy.float64'>'
	with 16507 stored elements in Compressed Sparse Row format>

In [35]:
predict_phishing("Congratulations! You've been selected to win a $1,000 gift card for Walmart! Click the link below to claim your prize: [malicious link]")

'Smishing'

In [37]:
predict_phishing("Hey jamnal , bro im your cousin i have been caught by traffic police , can you venmo me 50$ really quick? https://paytm/payl?q=1102")

'ham'

In [38]:
predict_phishing("Hello im calling from mumbai police station , your son has been arrested under drug case , we can release if u pay the money at http://bit.ly/?redirect=500$dollar.com , vnemo us 500$ really quick")

'Smishing'

In [39]:
import joblib

# Save the trained model
joblib.dump(model, 'phishing_detector_model.pkl')

# Save the TF-IDF vectorizer
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [40]:
import joblib

# Load the trained model
model = joblib.load('phishing_detector_model.pkl')

# Load the TF-IDF vectorizer
tfidf = joblib.load('tfidf_vectorizer.pkl')


In [41]:
def predict_phishing(input_message):
    input_vector = tfidf.transform([input_message])
    
    prediction = model.predict(input_vector)
    
    return prediction[0]  # Return the first element of the prediction array


In [ ]:
from flask import Flask, request, jsonify
import joblib

model = joblib.load('phishing_detector_model.pkl')
tfidf = joblib.load('tfidf_vectorizer.pkl')

app = Flask(__name__)

def predict_phishing(input_message):
    input_vector = tfidf.transform([input_message])
    prediction = model.predict(input_vector)
    return prediction[0]

@app.route('/predict', methods=['GET'])
def predict():
    # data = request.get_json()  # Get the JSON data from the request
    # message = data.get('message')  # Extract the message from the JSON data
    phishing_messages = [
        "Congratulations! You've been selected to win a $1,000 gift card for Walmart! Click the link below to claim your prize: [malicious link]",
        "Dear Customer, your account has been temporarily suspended due to suspicious activity. Please verify your identity by clicking here: [malicious link]",
        "Your device has been infected with a virus. Call our support team immediately at 1-800-123-4567 to resolve the issue and secure your data.",
        "Your payment is overdue! If you do not pay within 24 hours, your account will be suspended. Pay now: [malicious link]",
        "Your package is on hold due to an address issue. Please confirm your address here: [malicious link] to ensure timely delivery."
    ]
    
    result = []

    for message in phishing_messages:
        if message:
            output = predict_phishing(message)
            result.append({'message': message, 'prediction': output})  # Corrected the typo here
        else:
            result.append({'error': 'No message provided'})  # Handle missing message

    return jsonify({'result': result}), 200  # Corrected the return format

if __name__ == '__main__':
    try:
        print(flask.__version__)
    except Exception as e:
        print(f"Flask not found {e}")
    app.run(debug=False,host='0.0.0.0',port='5001')




Flask not found name 'flask' is not defined
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.120.97.134:5001
Press CTRL+C to quit
127.0.0.1 - - [26/Mar/2025 11:22:41] "GET /predict HTTP/1.1" 200 -
